In [1]:
import pandas as pd

In [16]:
us11 = pd.read_csv('11_national.csv')
us_previous = pd.to_numeric(us11[1:]['HD01_VD02'][1])
us16 = pd.read_csv('16_national.csv')
us_latest = pd.to_numeric(us16[1:]['HD01_VD02'][1]) 
nj11 = pd.read_csv('11_state.csv')
nj_previous =pd.to_numeric(nj11[1:]['HD01_VD02'][1])
nj16 = pd.read_csv('16_state.csv')
nj_latest = pd.to_numeric(nj16[1:]['HD01_VD02'][1]) 

In [18]:
nj_latest

39.5

In [41]:
muni16 = pd.read_csv('16_muni.csv', 
                     na_values=['-', '**', '***'], 
                     usecols = [ 'GEO.id', 'GEO.id2','GEOdisplaylabel_long', 
                           'median_age_latest','median_age_latest_moe' ],
                     header=None,
                     skiprows=2,
                    names=[ 'GEO.id', 'GEO.id2','GEOdisplaylabel_long', 
                           'median_age_latest','median_age_latest_moe' ])
cty16 = pd.read_csv('16_county.csv',
                    na_values=['-', '**','***'],
                   usecols = [ 'GEO.id', 'GEO.id2','GEOdisplaylabel_long', 
                           'median_age_latest','median_age_latest_moe' ],
                    header=None,
                    skiprows=2,
                   names=[ 'GEO.id', 'GEO.id2','GEOdisplaylabel_long', 
                           'median_age_latest','median_age_latest_moe' ])

age16 = muni16.append(cty16).reset_index().drop(labels='index', axis=1)

In [46]:
muni11 = pd.read_csv('11_muni.csv', 
                     na_values=['-', '**', '***'], 
                     usecols = [ 'GEO.id', 'GEO.id2','GEOdisplaylabel_long', 
                           'median_age_prev','median_age_prev_moe' ],
                     header=None,
                     skiprows=2,
                    names=[ 'GEO.id', 'GEO.id2','GEOdisplaylabel_long', 
                           'median_age_prev','median_age_prev_moe' ])
cty11 = pd.read_csv('11_county.csv',
                    na_values=['-', '**','***'],
                   usecols = [ 'GEO.id', 'GEO.id2','GEOdisplaylabel_long', 
                           'median_age_prev','median_age_prev_moe' ],
                    header=None,
                    skiprows=2,
                   names=[ 'GEO.id', 'GEO.id2','GEOdisplaylabel_long', 
                           'median_age_prev','median_age_prev_moe' ])

age11 = muni11.append(cty11).reset_index().drop(labels='index', axis=1)

In [51]:
age16['GEOdisplaylabel_long'] = age16['GEOdisplaylabel_long'].replace(to_replace=', New Jersey', value='', regex=True)
age11['GEOdisplaylabel_long'] = age11['GEOdisplaylabel_long'].replace(to_replace=', New Jersey', value='', regex=True)


In [54]:
age_cols = pd.DataFrame(age16.GEOdisplaylabel_long.str.split(',',1).tolist(), columns = ['GEOdisplaylabel', 'County'])

In [62]:
age16 = pd.concat([age16, age_cols ], axis=1)
age16['County'] = age16['County'].replace(to_replace='County', value='', regex=True)
age16.head()

,GEO.id,GEO.id2,GEOdisplaylabel_long,median_age_latest,median_age_latest_moe,GEOdisplaylabel,County
0,0600000US3400100000,3400100000,"County subdivisions not defined, Atlantic County",NaN,NaN,County subdivisions not defined,Atlantic
1,0600000US3400100100,3400100100,"Absecon city, Atlantic County",46.0,1.7,Absecon city,Atlantic
2,0600000US3400102080,3400102080,"Atlantic City city, Atlantic County",35.7,1.3,Atlantic City city,Atlantic
3,0600000US3400107810,3400107810,"Brigantine city, Atlantic County",53.5,2.6,Brigantine city,Atlantic
4,0600000US3400108680,3400108680,"Buena borough, Atlantic County",41.8,2.1,Buena borough,Atlantic


In [63]:
age = pd.merge(age16, age11, on='GEO.id2', how='left')

In [65]:
age.columns

Index([u'GEO.id_x', u'GEO.id2', u'GEOdisplaylabel_long_x',
       u'median_age_latest', u'median_age_latest_moe', u'GEOdisplaylabel',
       u'County', u'GEO.id_y', u'GEOdisplaylabel_long_y', u'median_age_prev',
       u'median_age_prev_moe'],
      dtype='object')

In [68]:
age_simp = age[[u'GEO.id_x', u'GEO.id2', u'GEOdisplaylabel', u'GEOdisplaylabel_long_x',u'County',
       u'median_age_latest', u'median_age_latest_moe', 
        u'median_age_prev',
       u'median_age_prev_moe']].rename(columns={'GEO.id_x':'GEO.id', 
                                                'GEOdisplaylabel_long_x':'GEOdisplaylabel_long'})

In [71]:
age_simp = age_simp[age_simp['GEOdisplaylabel'] != 'County subdivisions not defined']

In [72]:
age_simp.head()

,GEO.id,GEO.id2,GEOdisplaylabel,GEOdisplaylabel_long,County,median_age_latest,median_age_latest_moe,median_age_prev,median_age_prev_moe
1,0600000US3400100100,3400100100,Absecon city,"Absecon city, Atlantic County",Atlantic,46.0,1.7,45.7,1.4
2,0600000US3400102080,3400102080,Atlantic City city,"Atlantic City city, Atlantic County",Atlantic,35.7,1.3,35.7,1.2
3,0600000US3400107810,3400107810,Brigantine city,"Brigantine city, Atlantic County",Atlantic,53.5,2.6,48.4,2.8
4,0600000US3400108680,3400108680,Buena borough,"Buena borough, Atlantic County",Atlantic,41.8,2.1,35.1,4.0
5,0600000US3400108710,3400108710,Buena Vista township,"Buena Vista township, Atlantic County",Atlantic,38.9,3.9,45.1,2.5


In [73]:
age_simp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 586 entries, 1 to 590
Data columns (total 9 columns):
GEO.id                   586 non-null object
GEO.id2                  586 non-null int64
GEOdisplaylabel          586 non-null object
GEOdisplaylabel_long     586 non-null object
County                   565 non-null object
median_age_latest        586 non-null float64
median_age_latest_moe    586 non-null float64
median_age_prev          586 non-null float64
median_age_prev_moe      586 non-null float64
dtypes: float64(4), int64(1), object(4)
memory usage: 45.8+ KB


In [75]:
age_simp['age_moe_latest_per'] = (age_simp['median_age_latest_moe']/age_simp['median_age_latest'])*100 
age_simp['age_moe_prev_per'] = (age_simp['median_age_prev_moe']/age_simp['median_age_prev'])*100 

In [76]:
age_simp.head()

,GEO.id,GEO.id2,GEOdisplaylabel,GEOdisplaylabel_long,County,median_age_latest,median_age_latest_moe,median_age_prev,median_age_prev_moe,age_moe_latest_per,age_moe_prev_per
1,0600000US3400100100,3400100100,Absecon city,"Absecon city, Atlantic County",Atlantic,46.0,1.7,45.7,1.4,3.695652,3.063457
2,0600000US3400102080,3400102080,Atlantic City city,"Atlantic City city, Atlantic County",Atlantic,35.7,1.3,35.7,1.2,3.641457,3.361345
3,0600000US3400107810,3400107810,Brigantine city,"Brigantine city, Atlantic County",Atlantic,53.5,2.6,48.4,2.8,4.859813,5.785124
4,0600000US3400108680,3400108680,Buena borough,"Buena borough, Atlantic County",Atlantic,41.8,2.1,35.1,4.0,5.023923,11.396011
5,0600000US3400108710,3400108710,Buena Vista township,"Buena Vista township, Atlantic County",Atlantic,38.9,3.9,45.1,2.5,10.025707,5.543237


In [77]:
#Add NJ and Nationwide
age_simp['nj_median_age_latest'] = nj_latest
age_simp['nj_median_age_prev'] = nj_previous
age_simp['usa_median_age_latest'] = us_latest
age_simp['usa_median_age_prev'] = us_previous
age_simp['median_age_per_change'] = ((age_simp['median_age_latest'] - age_simp['median_age_prev']) / age_simp['median_age_prev']) * 100
age_simp['nj_median_age_change'] = ((age_simp['nj_median_age_latest'] - age_simp['nj_median_age_prev']) / age_simp['nj_median_age_prev']) * 100
age_simp['us_median_age_change'] = ((age_simp['usa_median_age_latest'] - age_simp['usa_median_age_prev']) / age_simp['usa_median_age_prev']) * 100

In [78]:
age_simp.head()

,GEO.id,GEO.id2,GEOdisplaylabel,GEOdisplaylabel_long,County,median_age_latest,median_age_latest_moe,median_age_prev,median_age_prev_moe,age_moe_latest_per,age_moe_prev_per,nj_median_age_latest,nj_median_age_prev,usa_median_age_latest,usa_median_age_prev,median_age_per_change,nj_median_age_change,us_median_age_change
1,0600000US3400100100,3400100100,Absecon city,"Absecon city, Atlantic County",Atlantic,46.0,1.7,45.7,1.4,3.695652,3.063457,39.5,38.7,37.7,37.0,0.656455,2.067183,1.891892
2,0600000US3400102080,3400102080,Atlantic City city,"Atlantic City city, Atlantic County",Atlantic,35.7,1.3,35.7,1.2,3.641457,3.361345,39.5,38.7,37.7,37.0,0.000000,2.067183,1.891892
3,0600000US3400107810,3400107810,Brigantine city,"Brigantine city, Atlantic County",Atlantic,53.5,2.6,48.4,2.8,4.859813,5.785124,39.5,38.7,37.7,37.0,10.537190,2.067183,1.891892
4,0600000US3400108680,3400108680,Buena borough,"Buena borough, Atlantic County",Atlantic,41.8,2.1,35.1,4.0,5.023923,11.396011,39.5,38.7,37.7,37.0,19.088319,2.067183,1.891892
5,0600000US3400108710,3400108710,Buena Vista township,"Buena Vista township, Atlantic County",Atlantic,38.9,3.9,45.1,2.5,10.025707,5.543237,39.5,38.7,37.7,37.0,-13.747228,2.067183,1.891892


In [79]:
age_simp.to_csv('town_by_town_median_age.csv')